<a href="https://colab.research.google.com/github/siliconemonster/AD-TrabalhoFinal/blob/main/Simula%C3%A7%C3%A3o_AD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Importando bibliotecas

In [1]:
import random
from random import sample
from random import random
from matplotlib import pyplot as plt
from matplotlib.legend_handler import HandlerLine2D
from collections import Counter
import sympy as sy
from numpy.linalg import matrix_power
from fractions import Fraction
import math
from math import *
import numpy as np
from numpy import random
import pandas as pd
import queue 
from scipy.stats import poisson

<h2> I. Simulação

In [ ]:
def initializeCache():  
  cache = sample([1,2,3], k= 2)
  return cache

In [ ]:
def cacheReceivesReq(caso, cache, requisicao):
  RequicaoAtendida = 0
  #print("Conteudo Requisicao: " + str(requisicao))
  #print("Cache:" + str(cache))

  if (requisicao in cache):
    if(caso == "FIFO" or caso == "Random" or caso == "Estatica"):
      cache = cache
      RequicaoAtendida = 1    
      #print("Cache possui requisção!") 

    elif (caso == "LRU"):
      RequicaoAtendida = 1    
      #print("Cache possui requisição!")

      cache.remove(requisicao)
      cache.insert(0, requisicao)
      #print("Novo estado da cache: " + str(cache)) 
   
  else:
    if (caso == "FIFO" or caso == "LRU"):
      cache.insert(0, requisicao)
      del cache[-1]
      #print("Novo estado da cache: " + str(cache)) 

    elif (caso == "Random"):
      cache.insert(0, requisicao)
      del cache[random.randint(-2,0)]
      #print("Novo estado da cache: " + str(cache)) 

  return cache, RequicaoAtendida

In [ ]:
def simulacaoCenario1(tipo, trials): 
    
  mediaSucessos = []

  for i in range(trials):

    queueCache1 = {}
    queueCache2 = {}

    sucesso = 0
    mediaSucesso = 0  

    cache1 = initializeCache()
    cache2 = initializeCache()

    reqAtendidasC1 = []
    reqAtendidasC2 = []


    t = 0
    max = 1000
    while (t < max):

      RequicaoAtendida = 0
      requisicao = random.randint(1,4)

      #print("Conteudo Requisitado: " + str(requisicao))
      #print("Cache 1: " + str(cache1))
      #print("Cache 2: " + str(cache2))


      TempoC1 = np.random.poisson(1/3,1)
      requisicaoTempoC1 = t + TempoC1[0]
      TempoC2 = np.random.poisson(1/3,1)
      requisicaoTempoC2 = t + TempoC2[0]      


      # Inserindo a requisição na fila
      queueCache1["req" + str(t)] = [requisicaoTempoC1,requisicao]
      queueCache2["req" + str(t)] = [requisicaoTempoC2,requisicao]
      
      # OrdenandO a Fila
      queueCache1 = {k: v for k, v in sorted(queueCache1.items(), key=lambda item: item[1])}
      queueCache2 = {k: v for k, v in sorted(queueCache2.items(), key=lambda item: item[1])}

      #print("Fila cache 1:" + str(queueCache1))
      #print("Fila cache 2:" + str(queueCache2))

      while (1):

        if (len(queueCache1)>0):
          primRequisicao = list(queueCache1.keys())[0]
          requisicao = queueCache1[primRequisicao]
          conteudoReq = requisicao[1]          
          tempoReeq = requisicao[0]

          if (tempoReeq == t):
            del queueCache1[primRequisicao]
            #print(str(primRequisicao) + " vai para a cache 1")
            cache1, RequicaoAtendida = cacheReceivesReq(tipo, cache1, conteudoReq)
            if (RequicaoAtendida):
              reqAtendidasC1.append(primRequisicao)
              #print("Requisição atendida")
            #else:
              #print("Requisição não atendida")


          else: break

        else: break

      while (1):

        if(len(queueCache2)>0):

          primRequisicao = list(queueCache2.keys())[0]
          requisicao = queueCache2[primRequisicao]
          conteudoReq = requisicao[1]
          tempoReeq = requisicao[0]

          if (tempoReeq == t):
            del queueCache2[primRequisicao]
            #print(str(primRequisicao) + " vai para a cache 2")
            cache2, RequicaoAtendida = cacheReceivesReq(tipo, cache2, conteudoReq)
            if (RequicaoAtendida):
              reqAtendidasC2.append(primRequisicao)
              #print("Requisição atendida")
            #else:
              #print("Requisição não atendida")

          else: break
        
        else: break

      #if(RequicaoAtendida):
          #print("Sucesso!")

      t = t + 1

      
    #print(reqAtendidasC1)
    #print(reqAtendidasC2)
    reqAtendidas = list(set(reqAtendidasC1).union(set(reqAtendidasC2)))
    #print(reqAtendidas)
    sucessos = len(reqAtendidas)
    #print("Sucessos = " + str(sucessos))
    mediaSucessos.append(sucessos/t)
    
  
  print("Media Sucessos = " + str(mediaSucessos))

    
  return mediaSucessos


In [ ]:
simulacaoCenario1("LRU", 3)

Media Sucessos = [0.721, 0.723, 0.722]


[0.721, 0.723, 0.722]

In [ ]:
random.seed(8)
tipo = "FIFO"

queueCache1 = {}
queueCache2 = {}

sucesso = 0
mediaSucesso = 0  

cache1 = initializeCache()
cache2 = initializeCache()
t = 0
max = 10
while (t < max):

  RequicaoAtendida = 0
  requisicao = random.randint(1,4)

  print("Conteudo Requisitado: " + str(requisicao))
  print("Cache 1: " + str(cache1))
  print("Cache 2: " + str(cache2))


  TempoC1 = np.random.poisson(1/3,1)
  requisicaoTempoC1 = t + TempoC1[0]
  TempoC2 = np.random.poisson(1/3,1)
  requisicaoTempoC2 = t + TempoC2[0]      


  # Inserindo a requisição na fila
  queueCache1["req" + str(t)] = [requisicaoTempoC1,requisicao]
  queueCache2["req" + str(t)] = [requisicaoTempoC2,requisicao]
  
  # OrdenandO a Fila
  queueCache1 = {k: v for k, v in sorted(queueCache1.items(), key=lambda item: item[1])}
  queueCache2 = {k: v for k, v in sorted(queueCache2.items(), key=lambda item: item[1])}

  print("Fila cache 1:" + str(queueCache1))
  print("Fila cache 2:" + str(queueCache2))

  while (1):

    if (len(queueCache1)>0):
      primRequisicao = list(queueCache1.keys())[0]
      requisicao = queueCache1[primRequisicao]
      conteudoReq = requisicao[1]
      tempoReeq = requisicao[0]

      if (tempoReeq == t):
        del queueCache1[primRequisicao]
        print(str(primRequisicao) + " vai para a cache 1")
        cache1, RequicaoAtendida = cacheReceivesReq(tipo, cache1, conteudoReq)
        if (RequicaoAtendida):
          print("Requisição atendida")
        else:
          print("Requisição não atendida")


      else: break

    else: break

  while (1):

    if(len(queueCache2)>0):

      primRequisicao = list(queueCache2.keys())[0]
      requisicao = queueCache2[primRequisicao]
      conteudoReq = requisicao[1]
      tempoReeq = requisicao[0]

      if (tempoReeq == t):
        del queueCache2[primRequisicao]
        print(str(primRequisicao) + " vai para a cache 2")
        cache2, RequicaoAtendida = cacheReceivesReq(tipo, cache2, conteudoReq)
        if (RequicaoAtendida):
          print("Requisição atendida")
        else:
          print("Requisição não atendida")

      else: break
    
    else: break

  if(RequicaoAtendida):
      print("Sucesso!")
      sucesso = sucesso + 1  

  t = t + 1
  print("")

Conteudo Requisitado: 1
Cache 1: [1, 3]
Cache 2: [2, 1]
Fila cache 1:{'req0': [2, 1]}
Fila cache 2:{'req0': [0, 1]}
req0 vai para a cache 2
Requisição atendida
Sucesso!

Conteudo Requisitado: 1
Cache 1: [1, 3]
Cache 2: [2, 1]
Fila cache 1:{'req1': [1, 1], 'req0': [2, 1]}
Fila cache 2:{'req1': [2, 1]}
req1 vai para a cache 1
Requisição atendida
Sucesso!

Conteudo Requisitado: 1
Cache 1: [1, 3]
Cache 2: [2, 1]
Fila cache 1:{'req0': [2, 1], 'req2': [3, 1]}
Fila cache 2:{'req1': [2, 1], 'req2': [2, 1]}
req0 vai para a cache 1
Requisição atendida
req1 vai para a cache 2
Requisição atendida
req2 vai para a cache 2
Requisição atendida
Sucesso!

Conteudo Requisitado: 3
Cache 1: [1, 3]
Cache 2: [2, 1]
Fila cache 1:{'req2': [3, 1], 'req3': [4, 3]}
Fila cache 2:{'req3': [3, 3]}
req2 vai para a cache 1
Requisição atendida
req3 vai para a cache 2
Requisição não atendida

Conteudo Requisitado: 1
Cache 1: [1, 3]
Cache 2: [3, 2]
Fila cache 1:{'req3': [4, 3], 'req4': [5, 1]}
Fila cache 2:{'req4': [4, 1

In [ ]:
len(queueCache1)


0

In [ ]:
primRequisicao[1]

'e'

In [ ]:
list(queueCache1.keys())[4]

IndexError: ignored

In [ ]:
def simulacaoCenario2(tipo, trials):
  queueCache1 = {}
  queueCache2 = {}
  sucesso = 0
  mediaSucesso = 0  
  cache1 = initializeCache()
  cache2 = initializeCache()
  usermiss = 0

  
  for i in range (trials):
    RequicaoAtendida = 0

    canal1 = random.rand()
    canal1falhou = 1
    sucessoc1 = 0
    requisicao = random.randint(1,4) 

    #print("\n Requisição: " + str(requisicao))
    #print("Cache 1: " + str(cache1))
    #print("Cache 2: " + str(cache2))

    if(canal1 <= 0.9):
      cache1, RequicaoAtendida = cacheReceivesReq(tipo, cache1, requisicao)
    #else:
     # print("Canal Falhou")

    canal2 = random.rand()
    if(canal2 <= 0.9):        
      cache2, RequicaoAtendida = cacheReceivesReq(tipo, cache2, requisicao)
    #else:
     # print("Canal Falhou")

    if(RequicaoAtendida):
      sucesso = sucesso + 1

    else:
      usermiss = usermiss + 1
  print("Media Sucessos = " + str(sucesso/trials)) 
  print("Usermiss = " + str(usermiss))
  print("Media Usermiss = " + str(usermiss/trials))

In [ ]:
simulacaoCenario1("FIFO", 100000)

KeyboardInterrupt: ignored

In [ ]:
simulacaoCenario1("LRU", 100000)

In [ ]:
simulacaoCenario1("Random", 100000)

In [ ]:
simulacaoCenario2("FIFO", 100000)

In [ ]:
'''
casoFifoMarkov  = [[sy.Rational('2/3'), 0, 0, 0,sy.Rational('1/3'),0], 
    [0, sy.Rational('2/3'), sy.Rational('1/3'), 0,0,0],
    [0, 0, sy.Rational('2/3'), 0,0,sy.Rational('1/3')],
    [sy.Rational('1/3'), 0, sy.Rational('2/3'), 0,0,0],
    [0, 0, 0, sy.Rational('1/3'),sy.Rational('2/3'),0],
    [0, sy.Rational('1/3'), 0, 0,0,sy.Rational('2/3')]] '''

"\ncasoFifoMarkov  = [[sy.Rational('2/3'), 0, 0, 0,sy.Rational('1/3'),0], \n    [0, sy.Rational('2/3'), sy.Rational('1/3'), 0,0,0],\n    [0, 0, sy.Rational('2/3'), 0,0,sy.Rational('1/3')],\n    [sy.Rational('1/3'), 0, sy.Rational('2/3'), 0,0,0],\n    [0, 0, 0, sy.Rational('1/3'),sy.Rational('2/3'),0],\n    [0, sy.Rational('1/3'), 0, 0,0,sy.Rational('2/3')]] "

In [ ]:
matrix_power(markovCasoFIFO, 10000).round(decimals=2)

array([[0.33, 0.  , 0.  , 0.33, 0.33, 0.  ],
       [0.  , 0.33, 0.33, 0.  , 0.  , 0.33],
       [0.  , 0.33, 0.33, 0.  , 0.  , 0.33],
       [0.33, 0.  , 0.  , 0.33, 0.33, 0.  ],
       [0.33, 0.  , 0.  , 0.33, 0.33, 0.  ],
       [0.  , 0.33, 0.33, 0.  , 0.  , 0.33]])

In [ ]:
matrix_power(markovCasoLRU, 10000).round(decimals=2)

array([[0.17, 0.17, 0.17, 0.17, 0.17, 0.17],
       [0.17, 0.17, 0.17, 0.17, 0.17, 0.17],
       [0.17, 0.17, 0.17, 0.17, 0.17, 0.17],
       [0.17, 0.17, 0.17, 0.17, 0.17, 0.17],
       [0.17, 0.17, 0.17, 0.17, 0.17, 0.17],
       [0.17, 0.17, 0.17, 0.17, 0.17, 0.17]])

In [ ]:
matrix_power(markovCasoRandom, 10000).round(decimals=2)

array([[0.33, 0.33, 0.33],
       [0.33, 0.33, 0.33],
       [0.33, 0.33, 0.33]])

In [ ]:
qCache1 = {}

In [ ]:
qCache1 = 

SyntaxError: ignored

<h2> II. Parte Analítica

Por conta do modo que a função para encontrar a distribuição estacionária é feita, trataremos as matrizes de transição como arrays do numpy:

In [6]:
markovCasoFIFO = np.array([[2/3, 0, 0, 0,1/3,0], 
                           [0, 2/3, 1/3, 0,0,0],
                           [0, 0, 2/3, 0,0,1/3],
                           [1/3, 0,0,2/3,0,0],
                           [0, 0, 0, 1/3,2/3,0],
                           [0, 1/3, 0, 0,0,2/3]])

markovCasoLRU  = np.array([[1/3, 0, 1/3, 0,1/3,0], 
                           [0, 1/3, 1/3, 0,1/3,0],
                           [1/3, 0, 1/3, 0,0,1/3],
                           [1/3, 0,0,1/3,0,1/3],
                           [0, 1/3, 0, 1/3,1/3,0],
                           [0, 1/3, 0, 1/3,0,1/3]])

markovCasoRandom = np.array([[2/3, 1/6, 1/6], 
                             [1/6, 2/3, 1/6],
                             [1/6, 1/6, 2/3]])

# Função para calcular a distribuição estacionária de uma cadeia de Markov por meio de uma matriz de transição P
def dist_estacionaria(P):
  evals, evecs = np.linalg.eig(P.T) #We have to transpose so that Markov transitions correspond to right multiplying by a column vector.  np.linalg.eig finds right eigenvectors.
  evec1 = evecs[:,np.isclose(evals, 1)]
  evec1 = evec1[:,0]
  stationary = evec1 / evec1.sum()
  stationary = stationary.real # parte real (acha autovalores e autovetores complexos)

  print(stationary)

dist_estacionaria(markovCasoFIFO)

[0.33333333 0.         0.         0.33333333 0.33333333 0.        ]
